In [63]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [57]:
df = pd.read_pickle('./data/SA/books/labeled.csv')
X = np.array(df['bow']).tolist()
y = np.array(df['tag']).tolist()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.8, random_state=15)
gnb = GaussianNB()
NB = gnb.fit(X_train, y_train)
y_pred = NB.predict(X_val)

In [62]:
accuracy_score(y_val, y_pred)

0.594375

In [66]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_val)

In [67]:
accuracy_score(y_val, y_pred)

0.714375

In [70]:
df = pd.read_pickle('./data/SA/books/labeled.csv')
X = np.array(df['bool-bow']).tolist()
y = np.array(df['tag']).tolist()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.8, random_state=15)
gnb = GaussianNB()
NB = gnb.fit(X_train, y_train)
y_pred = NB.predict(X_val)

In [71]:
accuracy_score(y_val, y_pred)

0.59875

In [72]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_val)

In [73]:
accuracy_score(y_val, y_pred)

0.72375